Import Lib

In [160]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [161]:
load_dotenv()

True

In [162]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

##### Make the Model structure

In [163]:
class EvalutionSchema(BaseModel):
  feedback: str = Field(description="Deatiled feedback for the essay")
  score: int = Field(description="Score out of 10", ge=0,le=10)

In [164]:
structureModel = model.with_structured_output(EvalutionSchema)

In [165]:
essay = """ The Role of Artificial Intelligence in Today’s World

Artificial Intelligence (AI) has become one of the most transformative technologies of the modern era. From smartphones to healthcare systems, AI is embedded in almost every aspect of our daily lives, shaping the way we work, communicate, and solve problems.

One of the most noticeable impacts of AI is in automation. Machines and algorithms now perform tasks that once required human effort, such as customer service through chatbots, fraud detection in banking, and recommendation systems on platforms like YouTube, Amazon, and Netflix. These AI-driven tools save time, reduce errors, and improve efficiency.

In healthcare, AI is helping doctors diagnose diseases earlier and more accurately. For example, deep learning models can detect tumors in medical scans, and predictive analytics can suggest personalized treatments. Similarly, in education, AI-powered platforms are offering personalized learning experiences to students worldwide, making quality education more accessible.

AI is also reshaping industries like transportation with self-driving cars, agriculture with smart farming technologies, and finance with automated trading systems. At the same time, AI is empowering individuals through virtual assistants such as Siri, Alexa, and Google Assistant, making everyday life more convenient.

However, the rapid growth of AI also raises ethical concerns. Issues such as job displacement, data privacy, bias in algorithms, and the potential misuse of AI technologies highlight the need for responsible development and regulation. Society must ensure that AI is used for the greater good while minimizing risks.

In conclusion, AI is no longer just a futuristic concept but a present-day reality. It is revolutionizing industries, improving human lives, and opening new possibilities. With careful management and ethical use, AI can serve as a powerful tool to build a smarter, more efficient, and inclusive world.
"""

In [166]:
prompt = 'Evalute the language quality of the following essay and provide a feedback & assign a score out of 10 {essay}'
structureModel.invoke(prompt)

EvalutionSchema(feedback='The essay is well-written and easy to understand. The arguments are clear and concise, and the supporting evidence is strong. The essay is well-organized and flows smoothly from one point to another. The conclusion is well-written and summarizes the main points of the essay.', score=9)

### WorkFlow

#### 1. State

In [167]:
class UPSCeassyState(TypedDict):
  eassyInput:str
  COAFeedback:str
  DOAFeedback:str
  LanguageFeedback:str
  overallFeedback:str
  individualScore:Annotated[list[int],operator.add]
  avgScore:float

#### 2. Graph

##### 2.1 Creating the Node

In [168]:
def COAFeedback(state:UPSCeassyState)->UPSCeassyState:
  prompt = f"Evalute the clarity of thought quality of the following essay and provide a feedback & assign a score out of 10 {state['eassyInput']}"
  
  output = structureModel.invoke(prompt)
  return {'COAFeedback':output.feedback, 'individualScore':[output.score]}


In [169]:
def DOAFeedback(state:UPSCeassyState)->UPSCeassyState:
  prompt = f"Evalute the Depth of analysis quality of the following essay and provide a feedback & assign a score out of 10 {state['eassyInput']}"
  
  output = structureModel.invoke(prompt)
  return {'DOAFeedback':output.feedback, 'individualScore':[output.score]}


In [170]:
def LanguageFeedback(state:UPSCeassyState)->UPSCeassyState:
  prompt = f"Evalute the language quality of the following essay and provide a feedback & assign a score out of 10 {state['eassyInput']}"
  
  output = structureModel.invoke(prompt)
  return {'LanguageFeedback':output.feedback, 'individualScore':[output.score]}


In [171]:
def overallFeedback(state:UPSCeassyState)->UPSCeassyState:

   # summary feedback
   prompt = f"Based on the following feedback, create the summarize feedback of all language which have LanguageFeedback {state['LanguageFeedback']},\n the clarity of depth feedback is {state['COAFeedback']}\n & the last which is\n Depth of Analysis {state['DOAFeedback']}"
   
   overallFeedbackAns = model.invoke(prompt).content

   # avg calculate
   avgScore = sum(state['individualScore']) / len(state['individualScore']) if state['individualScore'] else 0

   return {'overallFeedback':overallFeedbackAns, 'avgScore':avgScore}

#### 2.2 Defining the Graph

In [172]:
graph = StateGraph(UPSCeassyState)

graph.add_node('COAFeedback', COAFeedback)
graph.add_node('DOAFeedback', DOAFeedback)
graph.add_node('LanguageFeedback', LanguageFeedback)
graph.add_node('overallFeedback', overallFeedback)

graph.add_edge(START, 'COAFeedback')
graph.add_edge(START, 'DOAFeedback')
graph.add_edge(START, 'LanguageFeedback')
graph.add_edge('COAFeedback', 'overallFeedback')
graph.add_edge('DOAFeedback', 'overallFeedback')
graph.add_edge('LanguageFeedback', 'overallFeedback')
graph.add_edge('overallFeedback', END)

workflow = graph.compile()

##### 2.3 Show the Workflow

In [ ]:
workflow

##### 2.3 Execute the Workflow

In [173]:
initalState = {
  'eassyInput':essay
}

workflow.invoke(initalState)

{'eassyInput': ' The Role of Artificial Intelligence in Today’s World\n\nArtificial Intelligence (AI) has become one of the most transformative technologies of the modern era. From smartphones to healthcare systems, AI is embedded in almost every aspect of our daily lives, shaping the way we work, communicate, and solve problems.\n\nOne of the most noticeable impacts of AI is in automation. Machines and algorithms now perform tasks that once required human effort, such as customer service through chatbots, fraud detection in banking, and recommendation systems on platforms like YouTube, Amazon, and Netflix. These AI-driven tools save time, reduce errors, and improve efficiency.\n\nIn healthcare, AI is helping doctors diagnose diseases earlier and more accurately. For example, deep learning models can detect tumors in medical scans, and predictive analytics can suggest personalized treatments. Similarly, in education, AI-powered platforms are offering personalized learning experiences t